In [27]:
import pandas as pd
import numpy as np

In [28]:
%run "S:\YandexDisk\code\tasks\EDA.ipynb"

In [65]:
%run "S:\YandexDisk\code\tasks\Preparation.ipynb"

In [30]:
%run "S:\YandexDisk\code\tasks\Cleaning.ipynb"

In [31]:
%run "S:\YandexDisk\code\tasks\Models.ipynb"

In [32]:
%run "S:\YandexDisk\code\tasks\Selection.ipynb"

In [33]:
%run "S:\YandexDisk\code\tasks\Tools.ipynb"

In [34]:
# user_ids = pd.read_csv('train.csv', chunksize=100000, usecols=['user_id'], low_memory=False)

# user_counts = None
# for chunk in user_ids:
#     if user_counts is None:
#         user_counts = chunk.user_id.value_counts()
#     else:
#         user_counts = user_counts.add(chunk.user_id.value_counts(), fill_value=0)

# user_counts = user_counts.to_dict()

In [35]:
dtypes = {
#     'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
#     'task_container_id': 'int16',
    'content_type_id': 'int8',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float64',
    'prior_question_had_explanation': 'float32',
}

In [36]:
raw_train = pd.read_csv('train.csv',
                        nrows=100000,                        
                        dtype=dtypes,
                        usecols=dtypes.keys(),
                       low_memory=False)

In [37]:
raw_lectures = pd.read_csv('lectures.csv')
reduce_mem_usage(raw_lectures)

100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 2039.04it/s]


Mem. usage decreased to  0.01 Mb (58.8% reduction)


In [38]:
raw_questions = pd.read_csv('questions.csv')
reduce_mem_usage(raw_questions)

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 2506.46it/s]


Mem. usage decreased to  0.18 Mb (65.0% reduction)


In [39]:
raw_lectures['type_of'] = raw_lectures['type_of'].replace(
    'solving question', 'solving_question')

In [40]:
questions = pd.get_dummies(raw_questions.fillna('-1'), columns=['part'],
                           dtype='int8')
lectures = pd.get_dummies(raw_lectures, columns=['tag', 'part', 'type_of',],
                          dtype='int8')

for k in range(188):
    questions['tag_'+str(k)] = np.int8(0)
    
for i in questions.index:
    for tag in questions.loc[i, 'tags'].split():
        if tag != '-1':
            questions.loc[i, 'tag_'+tag] = 1
questions.drop(columns='tags', inplace=True)

In [41]:
part_columns = [column for column in lectures.columns if column.startswith('part_')]
tag_l_columns = [column for column in lectures.columns if column.startswith('tag_')]
tag_q_columns = [column for column in questions.columns if column.startswith('tag_')]
types_of_columns = [column for column in lectures.columns if column.startswith('type_of_')]

In [42]:
reduce_mem_usage(questions)
reduce_mem_usage(lectures)

100%|██████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 5219.98it/s]


Mem. usage decreased to  2.58 Mb (0.0% reduction)


100%|██████████████████████████████████████████████████████████████████████████████| 163/163 [00:00<00:00, 5830.59it/s]


Mem. usage decreased to  0.07 Mb (0.0% reduction)


In [43]:
train_l = raw_train.loc[raw_train.content_type_id == 1].merge(
    lectures, how='left', left_on='content_id',
    right_on='lecture_id').drop(columns=['lecture_id'])

train = pd.concat([raw_train.loc[raw_train.content_type_id == 0], train_l
                   ]).sort_values(['timestamp', 'content_id'],
                                  ascending=True).reset_index(drop=True).drop(columns=['timestamp'])

train[types_of_columns] = train[types_of_columns].fillna(0).astype('int16')
train[tag_l_columns] = train[tag_l_columns].fillna(0).astype('int8')
train[part_columns] = train[part_columns].fillna(0).astype('int8')
del train_l

In [45]:
for col in tqdm(part_columns):
    train[[
        f'{col}_sum_concept', f'{col}_sum_intention',
        f'{col}_sum_solving_question', f'{col}_sum_starter'
    ]] = train.loc[train[col] == 1, types_of_columns + ['user_id']].groupby(
        'user_id', sort=False).cumsum()

train.drop(columns=part_columns, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 43.04it/s]


In [46]:
for col in tqdm(tag_l_columns):
    if (train[col] == 0).all():
        train[[
            f'{col}_sum_concept', f'{col}_sum_intention',
            f'{col}_sum_solving_question', f'{col}_sum_starter'
        ]] = 0
    else:
        train[[
            f'{col}_sum_concept', f'{col}_sum_intention',
            f'{col}_sum_solving_question', f'{col}_sum_starter'
        ]] = train.loc[train[col] == 1,
                       ['user_id'] + types_of_columns].groupby(
                           'user_id', sort=False).cumsum()

train.drop(columns=tag_l_columns + types_of_columns, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [00:23<00:00,  6.42it/s]


In [47]:
exp_columns = [column for column in train.columns if (column.startswith('part_') or column.startswith('tag_'))]

In [48]:
train[exp_columns] = train[['user_id']+exp_columns].groupby('user_id', sort=False).ffill()
train[exp_columns] = train[exp_columns].fillna(0).astype('int16')

In [49]:
train = train.loc[train.content_type_id == 0].merge(
    questions, how='left', left_on='content_id',
    right_on='question_id').reset_index(drop=True).drop(
            columns=['question_id', 'content_type_id', 'correct_answer'])

In [50]:
train[part_columns+tag_q_columns] = train[part_columns+tag_q_columns].fillna(0).astype('int8')

In [51]:
train[['current_question_elapsed_time',
       'current_question_had_explanation']] = train[['user_id', 'bundle_id', 'prior_question_elapsed_time',
           'prior_question_had_explanation'
       ]].groupby(['user_id', 'bundle_id'], sort=False)[['user_id', 'prior_question_elapsed_time',
           'prior_question_had_explanation']].tail(1).groupby('user_id', sort=False).shift(-1)

In [52]:
train[['current_question_elapsed_time',
       'current_question_had_explanation']] = train[[
           'user_id', 'current_question_elapsed_time',
           'current_question_had_explanation'
       ]].groupby('user_id', sort=False).bfill()

In [53]:
q_stats = train[[
    'content_id', 'answered_correctly', 'current_question_elapsed_time',
    'current_question_had_explanation'
]].groupby('content_id').mean()

q_stats.columns = ['q_mean_answer', 'q_mean_time', 'q_mean_explain']

In [54]:
for col in tqdm(part_columns):
    train[[
        #         f'{col}_mean_answer',
        f'{col}_mean_time',
        f'{col}_mean_explain'
    ]] = train.loc[train[col] == 1, [
        'user_id',
        #         'answered_correctly',
        'current_question_elapsed_time',
        'current_question_had_explanation'
    ]].groupby('user_id', sort=False)[[
        #         'answered_correctly',
        'current_question_elapsed_time',
        'current_question_had_explanation'
    ]].expanding().mean().groupby('user_id', sort=False).shift().reset_index(
        level=0, drop=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  3.15it/s]


In [55]:
for col in tqdm(tag_q_columns):
    if (train[col] == 0).all():
        train[[
            #         f'{col}_mean_answer',
            f'{col}_mean_time',
            f'{col}_mean_explain'
        ]] = 0
    else:
        train[[
            #         f'{col}_mean_answer',
            f'{col}_mean_time',
            f'{col}_mean_explain'
        ]] = train.loc[train[col] == 1, [
            'user_id',
            #         'answered_correctly',
            'current_question_elapsed_time',
            'current_question_had_explanation'
        ]].groupby('user_id', sort=False)[[
            #         'answered_correctly',
            'current_question_elapsed_time',
            'current_question_had_explanation'
        ]].expanding().mean().groupby('user_id',
                                      sort=False).shift().reset_index(
                                          level=0, drop=True)

train.drop(columns=[
    'current_question_elapsed_time', 'current_question_had_explanation'
],
           inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████| 188/188 [00:42<00:00,  4.38it/s]


In [56]:
exp_columns = [column for column in train.columns if (column.startswith('part_') or column.startswith('tag_'))]
exp_columns = list(set(exp_columns) - set(part_columns) - set(tag_q_columns))

In [57]:
train[exp_columns] = train[['user_id']+exp_columns].groupby('user_id', sort=False).ffill()
train[exp_columns] = train[exp_columns].fillna(0)

In [58]:
train = train.merge(q_stats, how='left', left_on='content_id', right_index=True)

In [59]:
train.fillna(0, inplace=True)

In [60]:
reduce_mem_usage(train)

100%|██████████████████████████████████████████████████████████████████████████████| 1226/1226 [00:26<00:00, 47.08it/s]


Mem. usage decreased to 190.83 Mb (55.9% reduction)


In [62]:
X = train.drop(columns=[
    'user_id',
#     'content_id',
#         'bundle_id',
    'answered_correctly'])
y = train.answered_correctly

In [66]:
X_minmax = scale_it(X, X.columns, 'minmax')
reduce_mem_usage(X_minmax)

100%|██████████████████████████████████████████████████████████████████████████████| 1224/1224 [01:10<00:00, 17.37it/s]


Mem. usage decreased to 233.34 Mb (68.2% reduction)


In [67]:
mm = Lin_Clf(['log', 'lsvm'])

In [68]:
mm.val(X_minmax, y)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

log validated in 62.375394105911255


Validate log:  50%|████████████████████████████▌                            | 1/2 [01:02<01:02, 62.38s/it, score=0.808]

lsvm validated in 27.61081552505493


Validate lsvm: 100%|████████████████████████████████████████████████████████| 2/2 [01:29<00:00, 45.00s/it, score=0.766]


log     0.8079
lsvm    0.7660
Name: val_scores, dtype: float64

In [69]:
bo = BO_Clf(X_minmax, y)

In [70]:
bo.fit('lgbm', 50)

[WARNING 01-04 17:56:55] ax.service.ax_client: Random seed set to 13. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 01-04 17:56:55] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 10 trials, GPEI for subsequent trials]). Iterations after 10 will take longer to generate due to  model-fitting.
Optimize lgbm:  10%|█████▎                                               | 5/50 [10:50<1:37:36, 130.14s/it, best=0.813]


KeyboardInterrupt: 

In [71]:
bo.best_estimators_

{'lgbm': LGBMClassifier(bagging_fraction=0.6105435998179018, bagging_frec=2,
                cross_entropy_lambda=0.48732934053987265,
                feature_fraction=0.7650565416552126,
                learning_rate=0.08847665538285673, min_data_in_leaf=42,
                min_sum_hessian_in_leaf=0.027315270206097517, n_estimators=1000,
                n_jobs=0, num_leaves=5, random_state=13, rank_xendcg=3,
                tree_learner='voting')}